Code by Marvin Kinz, m.kinz@stud.uni-heidelberg.de
# Clean up data created by UE4
Removes multiple occurences of the same vertices in the point clouds and moves folder of initial objects out of the gravity folders, as they stay the same.
Should be run AFTER initial Surface Capturing and before Downsampling or calculating chamfer distances.

In [ ]:
import shutil
import glob
from pathlib import Path 
import numpy as np

def MakeUnique(ShapeFolder):
    """
    Remove multiple occurrences of same point in point clouds generated by UE4. Folder structure right now adapted for "Datasets Cube Cone Octa 10 pieces Gravity up to 4000 for Sampling"
    ATTENTION: If this is done, initial capturing of the cut sides in SurfaceCapture.ipynb does not work anymore. So that has to be done before and afterwards the recapture function can be used.
    """

    def Unique(Folder):
        """
        Helper function to make unique and return indices of unique vertices to make other folders unique as well.
        """
        objects=glob.glob(Folder)
        Indices=[] #unique indices for each object
        for obj in objects:
            #make vertices unique
            X=np.loadtxt(obj)
            Xu,ind,inv=np.unique(X,axis=0,return_index=1,return_inverse=1)
            np.savetxt(f"{obj}",Xu)
            #redirect triangle indices of duplicate vertices to unique vertices
            name=f"{obj[:-3]}triangle"
            T=np.loadtxt(name)
            for x,t in enumerate(T):
                for y,ts in enumerate(t):
                    T[x,y]=inv[np.int(ts)]
            np.savetxt(f"{name}",T)
            Indices.append(ind)
        return Indices

    #Algorithm works in place, so it makes a backup before
    shutil.copytree(ShapeFolder,f"{ShapeFolder} Backup")  

    shapes=glob.glob(f"{ShapeFolder}/*")
    for shape in shapes:
        print(shape)
        gravities=glob.glob(f"{shape}/*")
        #Initial folder same for all gravities, thus move and only handle once
        shutil.copytree(f"{gravities[0]}/Initial",f"{shape}/Initial")
        #make initial folder data unique and store indices
        ind_grav=Unique(f"{shape}/Initial/*.xyz")
        for grav in gravities:
            #remove duplicate initial folders
            shutil.rmtree(f"{grav}/Initial")
            #use stored indices to make deformed folder data unique
            objects=glob.glob(f"{grav}/Deformed/*.xyz")
            for i,obj in enumerate(objects):
                N=np.loadtxt(obj)
                np.savetxt(obj,N[ind_grav[i]])
            #get unique indices for slices
            ind=Unique(f"{grav}/Slices/*.xyz")
            #use indices to make slices deformed unique
            objects=glob.glob(f"{grav}/Slices deformed/*.xyz")
            for i,obj in enumerate(objects):
                #make normals unique
                name=f"{obj[:-3]}normals"    
                N=np.loadtxt(name)
                np.savetxt(f"{name}",N[ind[i]])  
                #make vertices unique
                N=np.loadtxt(obj)
                np.savetxt(obj,N[ind[i]])

In [ ]:
MakeUnique("2 Deformation and Slicing/SimulationResults")